# Predicting continuous disorder

Try dataset from [SETH](https://www.biorxiv.org/content/10.1101/2022.06.23.497276v1.full.pdf) with different architectures.

### Data

We contacted Rostlab, and they released their 1174 entry training dataset. As validation set, we use 15% of that i.e. 176 sequences.
One issue is that they have an ignore index (999) but PyTorch's MSELoss does not support that currently. Therefore, we built this ourselves.

### First Results

The performance for all models was way below the papers reported scores (around 0.55 spearman and 0.7 AUROC). This could be due to the following reasons:

- The data distributions of train and val sets are too dissimilar => New random sets (`chezod_cnn_random_shuffle_data.out`). Result: Improved it to 0.64 spearman and 0.86 AUROC
- Half-precision could be at fault => Try full precision (`chezod_cnn_full_precision.out`). Result: Did not improve it
- Combine: Full precision and random data without DP (`chezod_cnn_random_full_precision.out`). Result: Same as with random data but half-precision
- Different metric computation => Run their SETH implementation over our val set. Result: Indeed, we were calculating it incorrectly. The padded values (999) should be excluded and for AUROC, a perfect match should not count as 0 but as 1.

Running SETH on Colab over our validation sets gave the following metrics:

- Val Set 1
   - Mean Masked AUROC: 0.876
   - Mean Masked Spearman: 0.558
- Val Set 2
   - Mean Masked AUROC: 0.902
   - Mean Masked Spearman: 0.7

For our CNN training, we got:

- Val Set 1 (`chezod_cnn_corrected_scoring_set1.out`)
   - Mean Masked AUROC: 0.87
   - Mean Masked Spearman: 0.64
- Val Set 2 (`chezod_cnn_corrected_scoring_set2.out`)
   - Mean Masked AUROC: 0.90
   - Mean Masked Spearman: 0.79

### Hyperparameter Optimization

Running 30 rounds to select best embedding, architecture (cnn, rnn, linear), and learning rates (`chezod_hyperparam.out`). First, we failed because the maximum length calculation was incorrect. Here are the hyperparameters:

In [ ]:
config = {
    "model_name": tune.choice(['facebook/esm-1b', 'Rostlab/prot_bert_bfd', 'Rostlab/prot_t5_xl_half_uniref50-enc']),
    "architecture": tune.choice(['rnn', 'cnn', 'linear']),
    "learning_rate": tune.loguniform(1e-5, 1e-2),
    "encoder_learning_rate": tune.loguniform(5e-6, 1e-2),
}